## 1. Initialize project
Create or use an existing project to scope the RAG scenario

In [ ]:
import digitalhub as dh
project = dh.get_or_create_project("rag-demo")

### 1.1. Prepare secrets
For the project to work two secrets are needed:
- ``HF_TOKEN``: to contain the HuggingFace token if a protected model should be downloaded from the HuggingFace
- ``PG_CONN_URL``: full PGVector DB URL (with username/password) to connect to the corresponding vector store. The value may be obtained from the platform configuration or a new DB may be created from KRM with the necessary extensions.

In [ ]:
secret = project.new_secret("HF_TOKEN", secret_value="<your value>")
pg_secret = project.new_secret("PG_CONN_URL", secret_value="<full-pg-connection-url>")

## 2. Deploy the supporting LLM for text generation
Here we use the ``huggingfaceserve`` runtime for deploying the LLM service. Specifically, we deploy LLama3-8b-instruct model, but any HuggingFace-compatible model should work here. Alternatively, ``kubeai`` deployment may be used.

In [ ]:
chat_func = project.new_function("chat",
                                    kind="huggingfaceserve",
                                    model_name="chatmodel",
                                    path="huggingface://meta-llama/meta-llama-3-8b-instruct")

We run the function passing the specific execution properties:
- ``profile``: execution profile for the node selection and resource usage (depends on the platform). In this example 1xa100 refers to 1 GPU of type A100.
- ``max_length``: length of the context window for the text generation.
- ``secrets``: list of secreets to pass to LLM. Needed if, e.g., HuggingFace token is used.


In [ ]:
chat_run = chat_func.run(action="serve",
                           profile="1xa100",
                           max_length="5000",
                           secrets=["HF_TOKEN"],
                           wait=True)

Obtain the URL of the deployed service:

In [ ]:
chat_service_url = chat_run.refresh().status.to_dict()["service"]["url"]

## 2. Deploy the supporting LLM for embeddings

Embedding models map discrete data, such as words, to numerical vectors, which are more convenient for analysis, yet can still represent relationships between objects.    
Here we use the ``huggingfaceserve`` runtime for deploying the embedding model service. Specifically, we deploy thenlper/gte-base, but any HuggingFace-compatible model should work here. Alternatively, ``kubeai`` deployment may be used.

In [ ]:
emb_func = project.new_function("emb",
                                kind="huggingfaceserve",
                                model_name="embmodel",
                                path="huggingface://thenlper/gte-base")

In [ ]:
emb_run = emb_func.run(action="serve",
                       huggingface_task="text_embedding",
                       wait=True)

Obtain the URL of the deployed service:

In [ ]:
embedding_service_url = emb_run.refresh().status.to_dict()["service"]["url"]

## 3. Process the relevant information and store embeddings in the Vector storage

In the RAG scenario a typical task is to store the supporting information into the Vector storage and use it later for the text generation. For this purpose, consider an example scenario, where the relevant information is first scrapped from a Web page URL and then stored into the platform using the provided PGVector storage. Two components are required:
- Embeddings processor that uses Open Inference Protocol of our Embedding model service:
  ```python
    hf_embeddings = HuggingFaceInferenceAPIEmbeddings(
        api_key="ignore",
        api_url=f"http://{os.environ["EMBEDDING_SERVICE_URL"]}/v1/models/embmodel:predict"
    )
    class CEmbeddings(HuggingFaceInferenceAPIEmbeddings):
        def embed_documents(self, docs):
            return hf_embeddings.embed_documents(docs)["predictions"]

    custom_embeddings = CEmbeddings(api_key="ignore")
  ```
- PGVector storage from the platform:
  ```python
    vector_store = PGVector(
        embeddings=custom_embeddings,
        collection_name="my_docs",
        connection=os.environ["PG_CONN_URL"],
    )
  ```

We define a ``src/embedding.py`` Python Job to obtain the document, create chunks, and store their embeddings using the storage.

In [ ]:
data_func = project.new_function("create_embeddings", 
                                   kind="python", 
                                   python_version="PYTHON3_10",
                                   code_src="src/embedding.py",
                                   handler="embed",
                                   requirements=["transformers==4.50.3", "psycopg_binary", "langchain-text-splitters", "langchain-community", "langgraph", "langchain-core", "langchain-huggingface", "langchain_postgres", "langchain[openai]"]
                                  )

In [ ]:
data_run = data_func.run(action="job", 
                         parameters={"url": "https://lilianweng.github.io/posts/2023-06-23-agent/"},
                         envs=[{"name": "EMBEDDING_SERVICE_URL", "value": embedding_service_url}],
                         secrets=["PG_CONN_URL"]
                        )

The results of the elaboration are stored in the corresponding database.

## 4. Create RAG application API
Once the components and data is in place we can create a LangChain-based application and expose it as API in the platform. This will use the chat model service, the vector database, and the serverless functionality.

We create and deploy the serverless function that interacts with LLM and uses the vector store for retrieval. It uses a simple LangChaing graph composed out of two steps: retrieval and generation. The result of the generation is returned by the API.

In [ ]:
serve_func = project.new_function(
    name="rag_service", 
    kind="python", 
    python_version="PYTHON3_10", 
    code_src="src/serve.py",     
    handler="serve",
    init_function="init",
    requirements=["transformers==4.50.3", "psycopg_binary", "langchain-text-splitters", "langchain-community", "langgraph", "langchain-core", "langchain-huggingface", "langchain_postgres", "langchain[openai]"]
)

In [ ]:
serve_run = serve_func.run(
    action="serve",
    envs=[
            {"name": "EMBEDDING_SERVICE_URL", "value": embedding_service_url},
            {"name": "CHAT_SERVICE_URL", "value": chat_service_url}
         ],
    secrets=["PG_CONN_URL"]
)

To test our API we make a call to the service endpoint providing a JSON with the example question:

In [ ]:
import requests

serve_service_url = serve_run.refresh().status.to_dict()["service"]["url"]

res = requests.post(f"http://{serve_service_url}",json={"question": "What is decomposition in LLM?"})
res.json()